In [1]:
import configparser

config = configparser.ConfigParser()
config.read('env/pinecone.conf')

api_key = config["DEFAULT"]["PINECONE_API_KEY"]
environment = config["DEFAULT"]["PINECONE_ENVIRONMENT"]
openai_api_key = config["DEFAULT"]["OPENAI_API_KEY"]

In [2]:
from pinecone import Pinecone, PodSpec

index_name = "llamaindex-rag-fs"

pc = Pinecone(api_key=api_key)

pc.delete_index(index_name)

pc.create_index(
    name=index_name, 
    dimension=1536, 
    metric="cosine", 
    spec=PodSpec(environment=environment, pod_type="p1.x1")
)

pinecone_index = pc.Index(index_name)

In [3]:
from llama_index.vector_stores.pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

### Load into Vector Store
Load in documents into the PineconeVectorStore.

In [4]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext

In [5]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

In [7]:
import os
os.environ["OPENAI_API_KEY"] = openai_api_key

In [8]:
splitter = SentenceSplitter(chunk_size=1024)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], storage_context=storage_context
)

Upserted vectors: 100%|██████████| 108/108 [00:01<00:00, 59.31it/s]


### Define Vector Retriever
Now we’re ready to define our retriever against this vector store to retrieve a set of nodes.

In [9]:
query_str = "Can you tell me about the key concepts for safety finetuning"

In [11]:
# 1. Generate a Query Embedding
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
query_embedding = embed_model.get_query_embedding(query_str)

In [12]:
# 2. Query the Vector Database
#  We show how to query the vector database with different modes: default, sparse, and hybrid.
#  We first construct a VectorStoreQuery and then query the vector db. 

# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
query_result

VectorStoreQueryResult(nodes=[TextNode(id_='17ca33c7-f5c6-4009-9d15-b3a2fe1aa413', embedding=[-0.0239962451, -0.00637488067, 0.0150889736, -0.0263565313, -0.0186013039, 0.0239821952, -0.0125109227, 0.0102349324, -0.0313861892, -0.0503808744, 0.0111762369, 0.0034877446, -0.00924445502, -0.00812050886, 0.000514556479, 0.00879487675, 0.0323415436, -0.00326646771, 0.0122439852, -0.00804323796, -0.0156930946, 0.00937792379, -0.0271432921, -0.0109092994, -0.00531415641, 0.00324715, 0.0310209058, -0.0317233726, -0.021003738, -0.00488916459, -0.000527288648, 0.00224789185, 0.00441851234, -0.0053246934, -0.00373712, -0.0142670879, 0.00757258525, 0.000400625227, 0.032088656, -0.019837644, 0.025288783, 0.0205541607, -0.00391273666, -0.0270168483, 0.000378014607, 0.0196971521, 0.0138104847, -0.0227318052, -0.0324539356, 0.0211582817, 0.0435248055, 0.011766308, -0.0195426084, -0.00181938743, -0.000676123716, 0.00781844836, 0.015412108, 0.00849984, 0.0123844789, 0.00146639813, -0.00636434369, 0.0097

In [13]:
# 3. Parse Result into a set of Nodes
#  The VectorStoreQueryResult returns the set of nodes and similarities. We construct a NodeWithScore object with this.

from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

from llama_index.core.response.notebook_utils import display_source_node

for node in nodes_with_scores:
    display_source_node(node, source_length=1000)

**Node ID:** 17ca33c7-f5c6-4009-9d15-b3a2fe1aa413<br>**Similarity:** 0.830120862<br>**Text:** advice). The attack vectors explored consist of psychological manipulation (e.g., authority manipulation),
logic manipulation (e.g., false premises), syntactic manipulation (e.g., misspelling), semantic manipulation
(e.g., metaphor), perspective manipulation (e.g., role playing), non-English languages, and others.
We then define best practices for safe and helpful model responses: the model should first address immediate
safety concerns if applicable, then address the prompt by explaining the potential risks to the user, and finally
provide additional information if possible. We also ask the annotators to avoid negative user experience
categories (see Appendix A.5.2). The guidelines are meant to be a general guide for the model and are
iteratively refined and revised to include newly identified risks.
4.2.2
Safety Supervised Fine-Tuning
In accordance with the established guidelines from Section 4.2.1, we gather prompts and demonstrations
of safe model responses from trained annotato...<br>

**Node ID:** 71522559-0c2a-4e55-aafe-2fec209c836e<br>**Similarity:** 0.808828652<br>**Text:** TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necess...<br>

In [15]:
# 4. Put this into a Retriever

from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List

class PineconeRetriever(BaseRetriever):
    """Retriever over a pinecone vector store."""

    def __init__(
        self,
        vector_store: PineconeVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(query_str)
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [16]:
retriever = PineconeRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [17]:
retrieved_nodes = retriever.retrieve(query_str)
for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** 17ca33c7-f5c6-4009-9d15-b3a2fe1aa413<br>**Similarity:** 0.830120862<br>**Text:** advice). The attack vectors explored consist of psychological manipulation (e.g., authority manipulation),
logic manipulation (e.g., false premises), syntactic manipulation (e.g., misspelling), semantic manipulation
(e.g., metaphor), perspective manipulation (e.g., role playing), non-English languages, and others.
We then define best practices for safe and helpful model responses: the model should first address immediate
safety concerns if applicable, then address the prompt by explaining the potential risks to the user, and finally
provide additional information if possible. We also ask the annotators to avoid negative user experience
categories (see Appendix A.5.2). The guidelines are meant to be a general guide for the model and are
iteratively refined and revised to include newly identified risks.
4.2.2
Safety Supervised Fine-Tuning
In accordance with the established guidelines from Section 4.2.1, we gather prompts and demonstrations
of safe model responses from trained annotato...<br>

**Node ID:** 71522559-0c2a-4e55-aafe-2fec209c836e<br>**Similarity:** 0.808828652<br>**Text:** TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necess...<br>

In [18]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [19]:
response = query_engine.query(query_str)

In [20]:
print(str(response))

The key concepts for safety fine-tuning include supervised safety fine-tuning, safety RLHF (Reinforcement Learning from Human Feedback), and safety context distillation. Supervised safety fine-tuning involves gathering adversarial prompts and safe demonstrations to align the model with safety guidelines. Safety RLHF integrates safety into the RLHF pipeline by training a safety-specific reward model and using challenging adversarial prompts for fine-tuning. Safety context distillation refines the RLHF pipeline by prefixing model responses with a safety preprompt to generate safer responses and then fine-tuning the model without the preprompt to distill the safety context into the model.
